In [1]:
import numpy as np
from collections import Counter
import pandas as pd

In [2]:
def calc_ent(data):
    labels = Counter(data[:,-1])
    values = np.array(list(labels.values()))
    values_prob = values / values.sum()
    ent = -(values_prob * np.log2(values_prob)).sum()
    return ent

In [3]:
def split_by_value(data, axis, value):
    data_f = data[data[:, axis]==value]
    data_r = np.delete(data_f, axis, 1)
    return data_r

In [4]:
def ID3_split(data):
    feat_size = data.shape[1] - 1
    base_ent = calc_ent(data)  
    best_info_gain = 0.0 
    best_feat = -1 
    for i in range(feat_size):
        values_on_feat = set(data[:, i])  
        new_ent = 0.0  
        for value in values_on_feat:
            sub_data_set = split_by_value(data, i, value)
            prob = sub_data_set.shape[0] * 1.0 / data.shape[0] 
            new_ent += prob * calc_ent(sub_data_set)  
        info_gain = base_ent - new_ent 
        if (info_gain > best_info_gain):
            best_info_gain = info_gain  
            best_feat = i  
    return best_feat  

In [5]:
def majority_vote(category_list):
    return Counter(category_list).most_common(1)[0][0]

In [6]:
def tree_build(data, features_ori):
    features = features_ori[:]
    category_list = data[:, -1]
    if Counter(category_list)[category_list[0]] == category_list.shape[0]: 
        return category_list[0]  
    if data.shape[0] == 1: 
        return majority_vote(category_list) 
    best_feature = ID3_split(data)  
    # print("best: ", bestFeat)
    best_feature_name = features[best_feature]
    tree_grow = {best_feature_name: {}} 
    del (features[best_feature])  
    values_on_feat = set(data[:, best_feature])
    for value in values_on_feat: 
        sub_features = features[:]
        tree_grow[best_feature_name][value] = tree_build(
            split_by_value(data, best_feature, value), sub_features)
    return tree_grow

In [7]:
def classify(trained_tree, feature_names, test_sample):
    feature_key = list(trained_tree.keys())[0]
    feature_value = trained_tree[feature_key]
    feature_index = feature_names.index(feature_key)
    classify_res = 'Unknown'
    for key in feature_value.keys():
        if test_sample[feature_index] == key:
            if isinstance(feature_value[key], dict):
                classify_res = classify(feature_value[key], feature_names, test_sample)
            else:
                classify_res = feature_value[key]
    return classify_res

In [8]:
def data_split(data):
    import bootstrapping as bs
    _label = np.zeros(data.shape[0])
    train, _, test, _ = bs.bootstrapping(data, _label)
    return train, test

In [9]:
def decision_tree(path):
    data_ori = pd.read_csv(path)
    train, test = data_split(data_ori)
    car_tree = tree_build(np.array(train), list(train.columns))
    features = list(test.columns)
    error_counts = 0
    tested_count = test.shape[0]
    for i in range(tested_count):
        i_info = list(test.iloc[i])
        i_data = i_info[:-1]
        i_label = i_info[-1]
        if not classify(car_tree, features, i_data)==i_label :
            error_counts += 1
        # print("error!, at ", i)

    err_rate = 1.0 * error_counts / test.shape[0]
    print("%d samples tested\nerror rate is %f\n  acc rate is %f" 
          % (tested_count, err_rate, 1.0 - err_rate))
    return car_tree

In [10]:
car_tree = decision_tree("car.csv")

395 samples tested
error rate is 0.108861
  acc rate is 0.891139
